##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

In [2]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-2.0-flash-exp' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IkhlcmUncyBhIGJsdXJiIGZvciBhbiB1cGNvbWluZyBzdGFydHVwIGV2ZW50LiBSZXdyaXRlIHRoaXMgaW4gYSBtb3JlIHByb2Zlc3Npb25hbCB0b25lIGludGVuZGVkIGZvciBhIGNvcnBvcmF0ZSBlbWFpbC4gXFxuXFxuXFxcIkNhbGxpbmcgYWxsIHRlY2gganVua2llcywgaW52ZXN0b3JzLCBhbmQgY3VyaW91cyBtaW5kcyEg8J+klvCfp6Dwn46JXFxuXFxuUmVhZHkgdG8gd2l0bmVzcyB0aGUgZnV0dXJlPyBXZSdyZSB0aHJvd2luZyBhIHBpdGNoIHBhcnR5IHdoZXJlIGJyaWxsaWFudCBzdGFydHVwcyB3aWxsIHNob3djYXNlIHRoZWlyIG1pbmQtYmxvd2luZyBBSSBjcmVhdGlvbnMuIFRoaW5rIGhlYWx0aGNhcmUgdGhhdCByZWFkcyB5b3VyIG1pbmQgKGFsbW9zdCEpLCBidXNpbmVzc2VzIHRoYXQgcnVuIHRoZW1zZWx2ZXMgKHdlIHdpc2ghKSwgYW5kIHJvYm90cyB0aGF0IG1ha2UgeW91IGNvZmZlZSAod2UgZGVmaW5pdGVseSBuZWVkISkuV2hldGhlciB5b3UncmUgYW4gaW52ZXN0b3IsIGEgYnVkZGluZyBlbnRyZXByZW5ldXIsIG9yIGp1c3Qgc29tZW9uZSB3aG8gd2FudHMgdG8gZ2VlayBvdXQgb3ZlciBBSSwgdGhpcyBldmVudCBpcyB5b3VyIGphbSEgQ29tZSB3YXRjaCB0aGUgcGl0Y2hlcywgbWluZ2xlIHdpdGggaW5kdXN0cnkgZXhwZXJ0cywgYW5kIGdldCBpbnNwaXJlZCBieSB0aGVzZSByb2Nrc3RhciBjb21wYW5pZXMuIE9oLCBhbmQgZGlkIHdlIG1lbnRpb24gZnJlZSBmb29kIGFuZCBkcmlua3M/IFJlZ2lzdGVyIG5vdyBiZWZvcmUgdGhlIHJvYm90cyB0YWtlIGFsbCB0aGUgc3BvdHMhIPCfmIlcXFwiIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiU3ViamVjdDogSW52aXRhdGlvbjogQUkgSW5ub3ZhdGlvbiBTaG93Y2FzZSAmIE5ldHdvcmtpbmcgRXZlbnRcXG5cXG5EZWFyIFtOYW1lXSxcXG5cXG5XZSBjb3JkaWFsbHkgaW52aXRlIHlvdSB0byBhdHRlbmQgYW4gZXhjbHVzaXZlIHNob3djYXNlIG9mIGN1dHRpbmctZWRnZSBBSSBpbm5vdmF0aW9ucyBvbiBbRGF0ZV0gYXQgW1RpbWVdIGF0IFtMb2NhdGlvbl0uICBUaGlzIGV2ZW50IHdpbGwgZmVhdHVyZSBwcm9taXNpbmcgc3RhcnR1cHMgcHJlc2VudGluZyB0aGVpciBncm91bmRicmVha2luZyB3b3JrIGluIGFydGlmaWNpYWwgaW50ZWxsaWdlbmNlIGFjcm9zcyB2YXJpb3VzIHNlY3RvcnMsIGluY2x1ZGluZyBoZWFsdGhjYXJlLCBidXNpbmVzcyBhdXRvbWF0aW9uLCBhbmQgbW9yZS5cXG5cXG5BdHRlbmRlZXMgd2lsbCBoYXZlIHRoZSBvcHBvcnR1bml0eSB0byB3aXRuZXNzIGNvbXBlbGxpbmcgcGl0Y2hlcywgbmV0d29yayB3aXRoIGluZHVzdHJ5IGxlYWRlcnMgYW5kIGVudHJlcHJlbmV1cnMsIGFuZCBnYWluIHZhbHVhYmxlIGluc2lnaHRzIGludG8gdGhlIGZ1dHVyZSBvZiBBSS4gIFRoZSBldmVudCB3aWxsIGluY2x1ZGUgYW1wbGUgbmV0d29ya2luZyB0aW1lLCByZWZyZXNobWVudHMsIGFuZCBvcHBvcnR1bml0aWVzIHRvIGVuZ2FnZSBkaXJlY3RseSB3aXRoIHRoZSBwcmVzZW50aW5nIGNvbXBhbmllcy5cXG5cXG5UaGlzIGV2ZW50IGlzIGlkZWFsIGZvciBpbnZlc3RvcnMgc2Vla2luZyBwcm9taXNpbmcgdmVudHVyZXMsIGVudHJlcHJlbmV1cnMgaW50ZXJlc3RlZCBpbiBleHBsb3JpbmcgQUkgYXBwbGljYXRpb25zLCBhbmQgdGVjaG5vbG9neSBwcm9mZXNzaW9uYWxzIGtlZW4gdG8gbGVhcm4gYWJvdXQgdGhlIGxhdGVzdCBkZXZlbG9wbWVudHMgaW4gdGhlIGZpZWxkLlxcblxcblBsZWFzZSByZWdpc3RlciB5b3VyIGF0dGVuZGFuY2UgYnkgW1JlZ2lzdHJhdGlvbiBEZWFkbGluZV0gdmlhIHRoZSBmb2xsb3dpbmcgbGluazogW1JlZ2lzdHJhdGlvbiBMaW5rXVxcblxcbldlIGxvb2sgZm9yd2FyZCB0byB3ZWxjb21pbmcgeW91LlxcblxcblxcblNpbmNlcmVseSxcXG5cXG5UaGUgW1lvdXIgT3JnYW5pemF0aW9uIE5hbWVdIFRlYW1cXG4ifV19LHsicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IkhvdyBkbyBJIGltcHJvdmUgbXkgd3JpdGluZyBza2lsbHM/IEV4cGxhaW4gaW4gZGVwdGggYW5kIGdpdmUgZGFpbHkgaW5ub3ZhdGl2ZSBhbmQgY3JlYXRpdmUgcHJhY3RpY2UgaW5jbHVkaW5nIHRoZSByZXNvdXJjZXMuIEFsc28sIGJlZm9yZSB5b3UgZG8gdGhhdCwgZXhwbGFpbiB0aGUgd3JpdGluZyBzdHJ1Y3R1cmUgZm9yIHRoZSBtYWlsIGFib3ZlLiJ9XX0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6W3sidGV4dCI6Ik9rYXksIGxldCdzIGJyZWFrIHRoaXMgZG93bi4gRmlyc3QsIHdlJ2xsIGxvb2sgYXQgdGhlIHN0cnVjdHVyZSBvZiB0aGUgZW1haWwgSSBwcm92aWRlZCwgdGhlbiB3ZSdsbCBkaXZlIGRlZXAgaW50byBpbXByb3ZpbmcgeW91ciB3cml0aW5nIHNraWxscyB3aXRoIGNyZWF0aXZlIHByYWN0aWNlcyBhbmQgcmVzb3VyY2VzLlxcblxcbioqVW5kZXJzdGFuZGluZyB0aGUgRW1haWwgU3RydWN0dXJlKipcXG5cXG5UaGUgZW1haWwgYWJvdmUgZm9sbG93cyBhIGZhaXJseSBzdGFuZGFyZCwgcHJvZmVzc2lvbmFsIHN0cnVjdHVyZSBmb3IgYSBmb3JtYWwgaW52aXRhdGlvbiBvciBhbm5vdW5jZW1lbnQuIEhlcmUncyBhIGJyZWFrZG93biBvZiB0aGUga2V5IGVsZW1lbnRzIGFuZCB3aHkgdGhleSdyZSB1c2VkOlxcblxcbjEuICAqKlN1YmplY3QgTGluZToqKlxcbiAgICAqICAgKipQdXJwb3NlOioqIFRvIGltbWVkaWF0ZWx5IGNvbnZleSB0aGUgZW1haWwncyBwdXJwb3NlIGFuZCBlbmNvdXJhZ2UgcmVjaXBpZW50cyB0byBvcGVuIGl0LlxcbiAgICAqICAgKipFbGVtZW50czoqKiBDbGVhciwgY29uY2lzZSwgYW5kIHJlbGV2YW50LiBJdCBpbmNsdWRlcyBrZXl3b3JkcyBsaWtlIFxcXCJBSSxcXFwiIFxcXCJJbm5vdmF0aW9uLFxcXCIgYW5kIFxcXCJOZXR3b3JraW5nXFxcIiB0byBoaWdobGlnaHQgdGhlIGV2ZW50J3MgZm9jdXMuXFxuICAgICogICAqKldoeSBJdCBXb3JrczoqKiBBIGdvb2Qgc3ViamVjdCBsaW5lIGF2b2lkcyBiZWluZyB2YWd1ZSBhbmQgc2V0cyB0aGUgcmlnaHQgZXhwZWN0YXRpb25zLlxcblxcbjIuICAqKlNhbHV0YXRpb246KipcXG4gICAgKiAgICoqUHVycG9zZToqKiBUbyBwZXJzb25hbGl6ZSB0aGUgZW1haWwgYW5kIGNyZWF0ZSBhIHBvbGl0ZSBvcGVuaW5nLlxcbiAgICAqICAgKipFbGVtZW50czoqKiBcXFwiRGVhciBbTmFtZV1cXFwiIGlzIGEgZm9ybWFsIGdyZWV0aW5nIHN1aXRhYmxlIGZvciBhIHByb2Zlc3Npb25hbCBjb250ZXh0LiBUaGUgYnJhY2tldGVkIG5hbWUgaW5kaWNhdGVzIHBlcnNvbmFsaXphdGlvbi5cXG4gICAgKiAgICoqV2h5IEl0IFdvcmtzOioqIFVzaW5nIGEgbmFtZSAod2hlbiBwb3NzaWJsZSkgc2hvd3MgdGhhdCB5b3UgY2FyZSBhYm91dCB3aG8geW91J3JlIHdyaXRpbmcgdG8uXFxuXFxuMy4gICoqT3BlbmluZyBQYXJhZ3JhcGg6KipcXG4gICAgKiAgICoqUHVycG9zZToqKiBUbyBzdGF0ZSB0aGUgbWFpbiBwdXJwb3NlIG9mIHRoZSBlbWFpbCBpbW1lZGlhdGVseSBhbmQgY29uY2lzZWx5LlxcbiAgICAqICAgKipFbGVtZW50czoqKiBEaXJlY3RseSBzdGF0ZXMgdGhlIGludml0YXRpb24gdG8gdGhlIGV2ZW50LCBpbmNsdWRpbmcgZXNzZW50aWFsIGluZm9ybWF0aW9uIGxpa2UgZGF0ZSwgdGltZSwgYW5kIGxvY2F0aW9uLiBJdCBhbHNvIGhpZ2hsaWdodHMgdGhlIHNob3djYXNlIG9mIEFJIGlubm92YXRpb25zLlxcbiAgICAqICAgKipXaHkgSXQgV29ya3M6KiogR2V0cyBzdHJhaWdodCB0byB0aGUgcG9pbnQgYW5kIHByb3ZpZGVzIG5lY2Vzc2FyeSBkZXRhaWxzIHVwZnJvbnQuXFxuXFxuNC4gICoqQm9keSBQYXJhZ3JhcGgocyk6KipcXG4gICAgKiAgICoqUHVycG9zZToqKiBUbyBlbGFib3JhdGUgb24gdGhlIGV2ZW50LCBkZXNjcmliZSB3aGF0IGF0dGVuZGVlcyBjYW4gZXhwZWN0LCBhbmQgaGlnaGxpZ2h0IHRoZSBiZW5lZml0cyBvZiBhdHRlbmRpbmcuXFxuICAgICogICAqKkVsZW1lbnRzOioqIE1lbnRpb25zIHRoZSBkaWZmZXJlbnQgdHlwZXMgb2Ygc3RhcnR1cHMgcHJlc2VudGluZywgdGhlIG9wcG9ydHVuaXR5IHRvIG5ldHdvcmssIHRoZSBpbmNsdXNpb24gb2YgcmVmcmVzaG1lbnRzLCBhbmQgcG90ZW50aWFsIGJlbmVmaXRzIGZvciBkaWZmZXJlbnQgYXR0ZW5kZWVzIChpbnZlc3RvcnMsIGVudHJlcHJlbmV1cnMsIHRlY2ggcHJvZmVzc2lvbmFscykuXFxuICAgICogICAqKldoeSBJdCBXb3JrczoqKiBQcm92aWRlcyBtb3JlIGNvbnRleHQgYW5kIHBlcnN1YWRlcyB0aGUgcmVhZGVyIHRvIGF0dGVuZCBieSBvdXRsaW5pbmcgdmFsdWUgcHJvcG9zaXRpb25zLlxcblxcbjUuICAqKkNhbGwgdG8gQWN0aW9uIChDVEEpOioqXFxuICAgICogICAqKlB1cnBvc2U6KiogVG8gcHJvbXB0IHRoZSByZWFkZXIgdG8gdGFrZSBhIHNwZWNpZmljIGFjdGlvbi5cXG4gICAgKiAgICoqRWxlbWVudHM6KiogSW5jbHVkZXMgYSBjbGVhciBpbnN0cnVjdGlvbiAocmVnaXN0ZXIgeW91ciBhdHRlbmRhbmNlKSBhbG9uZyB3aXRoIHRoZSBkZWFkbGluZSBhbmQgcmVnaXN0cmF0aW9uIGxpbmsuXFxuICAgICogICAqKldoeSBJdCBXb3JrczoqKiBEaXJlY3RzIHRoZSByZWFkZXIgdG8gdGhlIG5leHQgc3RlcCB0aGV5IHNob3VsZCB0YWtlIGFuZCBwcm92aWRlcyB0aGUgbmVjZXNzYXJ5IHRvb2xzLlxcblxcbjYuICAqKkNsb3NpbmcgUGFyYWdyYXBoOioqXFxuICAgICogICAqKlB1cnBvc2U6KiogVG8gcG9saXRlbHkgZW5kIHRoZSBlbWFpbC5cXG4gICAgKiAgICoqRWxlbWVudHM6KiogQSBicmllZiwgZm9yd2FyZC1sb29raW5nIHN0YXRlbWVudCBpbmRpY2F0aW5nIGFudGljaXBhdGlvbi5cXG4gICAgKiAgICoqV2h5IEl0IFdvcmtzOioqIExlYXZlcyBhIHBvc2l0aXZlIGFuZCBwcm9mZXNzaW9uYWwgaW1wcmVzc2lvbi5cXG5cXG43LiAgKipDbG9zaW5nIGFuZCBTaWduYXR1cmU6KipcXG4gICAgKiAgICoqUHVycG9zZToqKiBUbyBlbmQgdGhlIGVtYWlsIGZvcm1hbGx5LlxcbiAgICAqICAgKipFbGVtZW50czoqKiBcXFwiU2luY2VyZWx5LFxcXCIgYW5kIHRoZSBzZW5kZXIncyBvcmdhbml6YXRpb24gbmFtZS5cXG4gICAgKiAgICoqV2h5IEl0IFdvcmtzOioqIFN0YW5kYXJkIGNsb3NpbmcgZm9yIGJ1c2luZXNzIGNvcnJlc3BvbmRlbmNlLCBhbmQgcHJvdmlkZXMgYWNjb3VudGFiaWxpdHkuXFxuXFxuKipLZXkgUHJpbmNpcGxlcyBpbiB0aGlzIFN0cnVjdHVyZToqKlxcblxcbiogICAqKkNsYXJpdHk6KiogQXZvaWRzIGphcmdvbiBhbmQgdXNlcyBzaW1wbGUgbGFuZ3VhZ2UuXFxuKiAgICoqQ29uY2lzZW5lc3M6KiogR2V0cyBzdHJhaWdodCB0byB0aGUgcG9pbnQgYW5kIGF2b2lkcyB1bm5lY2Vzc2FyeSB3b3Jkcy5cXG4qICAgKipQcm9mZXNzaW9uYWxpc206KiogTWFpbnRhaW5zIGEgZm9ybWFsIHRvbmUgYXBwcm9wcmlhdGUgZm9yIGJ1c2luZXNzIGNvbW11bmljYXRpb24uXFxuKiAgICoqRm9jdXM6KiogQ2xlYXJseSBjb21tdW5pY2F0ZXMgdGhlIGV2ZW50J3MgcHVycG9zZSBhbmQgdGhlIHZhbHVlIGl0IG9mZmVycy5cXG5cXG4qKkltcHJvdmluZyBZb3VyIFdyaXRpbmcgU2tpbGxzOiBJbi1EZXB0aCBHdWlkZSoqXFxuXFxuT2theSwgbm93IG9udG8gdGhlIG1haW4gY291cnNlIOKAkyBpbXByb3ZpbmcgeW91ciB3cml0aW5nIHNraWxscy4gVGhpcyBpcyBub3QgYSBxdWljayBmaXgsIGJ1dCBhIGpvdXJuZXkuIEhlcmUncyBhIG11bHRpLWZhY2V0ZWQgYXBwcm9hY2g6XFxuXFxuKioxLiBUaGUgRm91bmRhdGlvbjogVW5kZXJzdGFuZGluZyBHcmFtbWFyLCBWb2NhYnVsYXJ5LCBhbmQgU3R5bGUqKlxcblxcbiogICAqKkdyYW1tYXI6KipcXG4gICAgKiAgICoqUHJhY3RpY2U6KiogVXNlIG9ubGluZSB0b29scyBsaWtlIEdyYW1tYXJseSwgSGVtaW5nd2F5IEVkaXRvciwgb3IgUHJvV3JpdGluZ0FpZCAoc29tZSBhcmUgZnJlZSwgb3RoZXJzIGhhdmUgc3Vic2NyaXB0aW9uIG9wdGlvbnMpLiBGb2N1cyBvbiBvbmUgZ3JhbW1hdGljYWwgY29uY2VwdCBhdCBhIHRpbWUgKGUuZy4sIHN1YmplY3QtdmVyYiBhZ3JlZW1lbnQsIGNvcnJlY3QgdGVuc2UgdXNhZ2UpXFxuICAgICogICAqKlJlc291cmNlOioqICpUaGUgRWxlbWVudHMgb2YgU3R5bGUqIGJ5IFN0cnVuayBhbmQgV2hpdGUgKGEgY2xhc3NpYyBhbmQgY29uY2lzZSBndWlkZSkuXFxuKiAgICoqVm9jYWJ1bGFyeToqKlxcbiAgICAqICAgKipQcmFjdGljZToqKiBEYWlseSByZWFkIGEgdmFyaWV0eSBvZiBnZW5yZXMgYW5kIG5vdGUgZG93biBuZXcgd29yZHMgaW4gYSBub3RlYm9vay9hcHAsIExlYXJuIGEgbmV3IHdvcmQgZXZlcnkgZGF5LiBVc2UgZmxhc2hjYXJkIGFwcHMgbGlrZSBBbmtpIHRvIHF1aXogeW91cnNlbGYuXFxuICAgICogICAqKlJlc291cmNlOioqICpXb3JkIFBvd2VyIE1hZGUgRWFzeSogYnkgTm9ybWFuIExld2lzIGlzIGFuIGV4Y2VsbGVudCBib29rIGZvciB2b2NhYnVsYXJ5IGJ1aWxkaW5nLlxcbiogICAqKlN0eWxlOioqXFxuICAgICogICAqKlByYWN0aWNlOioqIEFuYWx5emUgZ29vZCB3cml0aW5nIHNhbXBsZXMgaW4gZGlmZmVyZW50IHN0eWxlcyAoZS5nLiwgZm9ybWFsLCBpbmZvcm1hbCwgcGVyc3Vhc2l2ZSwgbmFycmF0aXZlKS4gVHJ5IHRvIGVtdWxhdGUgdGhlIHN0eWxlIHlvdSB3YW50IHRvIGxlYXJuIGFuZCBzZWUgd2hlcmUgeW91IGdvIHdyb25nLlxcbiAgICAqICAgKipSZXNvdXJjZToqKiAqT24gV3JpdGluZyBXZWxsKiBieSBXaWxsaWFtIFppbnNzZXIgKGEgcHJhY3RpY2FsIGd1aWRlIHRvIGNsZWFyIGFuZCBjb25jaXNlIG5vbi1maWN0aW9uIHdyaXRpbmcpLlxcblxcbioqMi4gRGFpbHkgQ3JlYXRpdmUgV3JpdGluZyBQcmFjdGljZXMqKlxcblxcblRoZXNlIGFyZSBkZXNpZ25lZCB0byBiZSBmdW4sIGRpdmVyc2UsIGFuZCBidWlsZCBkaWZmZXJlbnQgd3JpdGluZyBtdXNjbGVzLlxcblxcbiogICAqKk1vcm5pbmcgUGFnZXM6KipcXG4gICAgKiAgICoqQ29uY2VwdDoqKiAgV3JpdGUgdGhyZWUgcGFnZXMgb2YgZnJlZS1mbG93aW5nIHRob3VnaHRzIGV2ZXJ5IG1vcm5pbmcuIERvbid0IGNlbnNvciBvciBlZGl0LCBqdXN0IGxldCB5b3VyIG1pbmQgd2FuZGVyLlxcbiAgICAqICAgKipQdXJwb3NlOioqIERldmVsb3BzIGZsdWVuY3ksIGxvb3NlbnMgeW91ciB3cml0aW5nIFxcXCJtdXNjbGVzLFxcXCIgYW5kIGhlbHBzIHlvdSBkaXNjb3ZlciB5b3VyIHZvaWNlLlxcbiAgICAqICAgKipSZXNvdXJjZToqKiAqVGhlIEFydGlzdCdzIFdheSogYnkgSnVsaWEgQ2FtZXJvbiAoZXhwbGFpbnMgdGhpcyBwcmFjdGljZSBpbiBkZXRhaWwpLlxcbiogICAqKlNpeC1Xb3JkIFN0b3J5IENoYWxsZW5nZToqKlxcbiAgICAqICAgKipDb25jZXB0OioqIFRlbGwgYSBjb21wbGV0ZSBzdG9yeSBpbiBqdXN0IHNpeCB3b3Jkcy5cXG4gICAgKiAgICoqUHVycG9zZToqKiAgRm9yY2VzIHlvdSB0byBiZSBjb25jaXNlLCBjcmVhdGl2ZSwgYW5kIGltcGFjdGZ1bC5cXG4gICAgKiAgICoqUHJhY3RpY2U6KiogVHJ5IHRoaXMgaW4geW91ciBub3RlYm9vaywgYXMgYSBqb3VybmFsIHByb21wdCwgb3IgaW4gd3JpdGluZyBjb21tdW5pdGllcy5cXG4qICAgKipTdG9yeSBQcm9tcHRzOioqXFxuICAgICogICAqKkNvbmNlcHQ6KiogIEZpbmQgc3RvcnkgcHJvbXB0cyBvbmxpbmUgKFBpbnRlcmVzdCwgR29vZ2xlKS4gUGljayBvbmUgYW5kIHdyaXRlIGEgc2hvcnQgc3Rvcnkgb3Igc2NlbmUgYmFzZWQgb24gaXQuXFxuICAgICogICAqKlB1cnBvc2U6KiogIERldmVsb3BzIGNyZWF0aXZlIG5hcnJhdGl2ZSBza2lsbHMgYW5kIGdldHMgeW91IHRoaW5raW5nIG91dHNpZGUgdGhlIGJveC5cXG4gICAgKiAgICoqUmVzb3VyY2U6KiogIEEgc2ltcGxlIGdvb2dsZSBzZWFyY2ggd2lsbCB5aWVsZCBtYW55IHNpdGVzLiBDaGVjayBvdXQgXFxcIldyaXRpbmcgUHJvbXB0c1xcXCIgYnkgVGhvdWdodCBDYXRhbG9nIGFuZCBcXFwiQ3JlYXRpdmUgV3JpdGluZyBQcm9tcHRzXFxcIiBieSBKb3VybmFsIEJ1ZGRpZXMuXFxuKiAgICoqRGVzY3JpcHRpb24gRXhlcmNpc2VzOioqXFxuICAgICogICAqKkNvbmNlcHQ6KiogUGljayBhIHJhbmRvbSBvYmplY3QsIHBlcnNvbiwgb3IgcGxhY2UuIFNwZW5kIDUtMTAgbWludXRlcyB3cml0aW5nIGEgZGV0YWlsZWQgZGVzY3JpcHRpb24gdXNpbmcgYWxsIGZpdmUgc2Vuc2VzLlxcbiAgICAqICAgKipQdXJwb3NlOioqICBJbXByb3ZlcyB5b3VyIGFiaWxpdHkgdG8gcGFpbnQgdml2aWQgcGljdHVyZXMgd2l0aCB3b3JkcyBhbmQgbm90aWNlIHRoZSBmaW5lciBkZXRhaWxzLlxcbiAgICAqICAgKipQcmFjdGljZToqKiBTdGFydCBzbWFsbCwgZGVzY3JpYmUgYSBwaWVjZSBvZiBmcnVpdCwgdGhlbiBnbyB1cCB0byBsYXJnZXIgY29uY2VwdHMgbGlrZSB0aGUgc2t5LlxcbiogICAqKkpvdXJuYWxpbmc6KipcXG4gICAgKiAgICoqQ29uY2VwdDoqKiBLZWVwIGEgam91cm5hbCBhbmQgd3JpdGUgYWJvdXQgYW55dGhpbmcgLSB5b3VyIGRhaWx5IGV4cGVyaWVuY2VzLCB0aG91Z2h0cywgZmVlbGluZ3MsIHJlZmxlY3Rpb25zLCBldGMuXFxuICAgICogICAqKlB1cnBvc2U6KiogQnVpbGRzIHdyaXRpbmcgc3RhbWluYSBhbmQgY2FuIHVuY292ZXIgeW91ciB1bmlxdWUgd3JpdGluZyBzdHlsZS5cXG4gICAgKiAgICoqUmVzb3VyY2U6KiogIEFueSBub3RlYm9vayBvciBhcHAgd29ya3M7IHlvdSBjYW4gYWxzbyBkbyBhdWRpbyByZWNvcmRpbmcgam91cm5hbCBlbnRyaWVzLlxcbiogICAqKkltaXRhdGlvbi9QYXJvZHk6KipcXG4gICAgKiAgICoqQ29uY2VwdDoqKiBUYWtlIGEgcGllY2Ugb2Ygd3JpdGluZyB5b3UgYWRtaXJlIGFuZCBpbWl0YXRlIGl0cyBzdHlsZS4gT3Igd3JpdGUgYSBwYXJvZHkgdXNpbmcgdGhlIHRvbmUgb2YgYSBwaWVjZSB5b3UgZGlzbGlrZS5cXG4gICAgKiAgICoqUHVycG9zZToqKiBIZWxwcyB5b3UgZGVjb25zdHJ1Y3QgZ29vZCB3cml0aW5nIGFuZCB1bmRlcnN0YW5kIGhvdyBzdHlsZSBpcyBjcmVhdGVkLlxcbiAgICAqICAgKipQcmFjdGljZToqKiBDYW4gYmUgd2l0aCBhbnl0aGluZyAtIGVtYWlscywgYXJ0aWNsZXMsIG5vdmVscywgZXZlbiByZWNpcGVzLlxcbiogICAqKk9uZS1MaW5lciBDaGFsbGVuZ2U6KipcXG4gICAgKiAgICoqQ29uY2VwdDoqKiAgV3JpdGUgYSBjbGV2ZXIsIHdpdHR5LCBvciBpbnRyaWd1aW5nIG9uZS1saW5lciB0aGF0IGNhbiBzdGFuZCBhbG9uZS5cXG4gICAgKiAgICoqUHVycG9zZToqKiAgU2hhcnBlbnMgeW91ciBhYmlsaXR5IHRvIGJlIGNvbmNpc2UgYW5kIGNyZWF0ZSBhbiBpbXBhY3QgaW4gYSBzaG9ydCBhbW91bnQgb2YgdGltZS5cXG4gICAgKiAgICoqUHJhY3RpY2U6KiogVHJ5IGl0IG9uIHlvdXIgb3duLCBvciBwYXJ0aWNpcGF0ZSBpbiBvbmxpbmUgY2hhbGxlbmdlcyBpbiB3cml0aW5nIGNvbW11bml0aWVzLlxcblxcbioqMy4gUmVhZGluZyBXaWRlbHkgYW5kIEFjdGl2ZWx5KipcXG5cXG4qICAgKipDb25jZXB0OioqIFJlYWRpbmcgZGlmZmVyZW50IGtpbmRzIG9mIG1hdGVyaWFsIGlzIGxpa2UgY3Jvc3MtdHJhaW5pbmcgZm9yIHlvdXIgd3JpdGluZyBtdXNjbGVzLlxcbiogICAqKlByYWN0aWNlOioqIE1ha2UgYSBnb2FsIHRvIHJlYWQgYSBwYXJ0aWN1bGFyIGdlbnJlIHBlciB3ZWVrIG9yIG1vbnRoLlxcbiogICAqKkhvdzoqKiBOb3RlIGRvd24gcGFzc2FnZXMgdGhhdCByZXNvbmF0ZSB3aXRoIHlvdS4gQW5hbHl6ZSB3aHkgdGhleSBhcmUgZWZmZWN0aXZlLiBPYnNlcnZlIHRoZSBhdXRob3IncyB2b2NhYnVsYXJ5IGFuZCBzZW50ZW5jZSBzdHJ1Y3R1cmUuXFxuXFxuKio0LiBTZWVraW5nIEZlZWRiYWNrIGFuZCBSZXZpc2lvbioqXFxuXFxuKiAgICoqQ29uY2VwdDoqKiBObyB3cml0ZXIgaW1wcm92ZXMgaW4gaXNvbGF0aW9uLlxcbiogICAqKlByYWN0aWNlOioqIFNoYXJlIHlvdXIgd3JpdGluZyB3aXRoIHRydXN0ZWQgZnJpZW5kcywgY29sbGVhZ3Vlcywgb3Igb25saW5lIHdyaXRpbmcgZ3JvdXBzLiBCZSBvcGVuIHRvIGNvbnN0cnVjdGl2ZSBjcml0aWNpc20uXFxuKiAgICoqSG93OioqIEFzayBzcGVjaWZpYyBxdWVzdGlvbnMgYWJvdXQgeW91ciB3cml0aW5nOlxcbiAgICAqICAgSXMgdGhlIGxhbmd1YWdlIGNsZWFyP1xcbiAgICAqICAgSXMgdGhlIHN0cnVjdHVyZSBsb2dpY2FsP1xcbiAgICAqICAgSXMgdGhlIHRvbmUgYXBwcm9wcmlhdGU/XFxuXFxuKio1LiBVdGlsaXppbmcgVGVjaG5vbG9neSBFZmZlY3RpdmVseSoqXFxuXFxuKiAgICoqQXBwczoqKiBHcmFtbWFybHksIEhlbWluZ3dheSBFZGl0b3IsIFNjcml2ZW5lciwgUHJvV3JpdGluZ0FpZCwgRXZlcm5vdGUsIEdvb2dsZSBEb2NzXFxuKiAgICoqT25saW5lIENvbW11bml0aWVzOioqIFJlZGRpdCdzIHIvd3JpdGluZywgci9kZXN0cnVjdGl2ZXJlYWRlcnMsIGNyaXRpcXVlIGNpcmNsZSwgb25saW5lIHdyaXRpbmcgZ3JvdXBzLlxcbiogICAqKlRvb2xzOioqIFRoZXNhdXJ1c2VzLCBEaWN0aW9uYXJ5LCB0ZXh0LXRvLXNwZWVjaCBzb2Z0d2FyZSAoZm9yIHByb29mcmVhZGluZykuXFxuXFxuKipJbm5vdmF0aXZlICYgQ3JlYXRpdmUgUHJhY3RpY2UqKlxcblxcbkhlcmUgYXJlIGEgZmV3IGFkZGl0aW9uYWwgaW5ub3ZhdGl2ZSAmIGNyZWF0aXZlIHByYWN0aWNlcyB0byB3ZWF2ZSBpbnRvIHlvdXIgcm91dGluZTpcXG5cXG4qICAgKipcXFwiRXhxdWlzaXRlIENvcnBzZVxcXCIgV3JpdGluZzoqKlxcbiAgICAqICAgKipDb25jZXB0OioqIEEgY29sbGFib3JhdGl2ZSB3cml0aW5nIGV4ZXJjaXNlIHdoZXJlIGVhY2ggcGVyc29uIGNvbnRyaWJ1dGVzIGEgc2VudGVuY2Ugb3IgcGFyYWdyYXBoIHdpdGhvdXQgc2VlaW5nIHdoYXQgdGhlIG90aGVycyBoYXZlIHdyaXR0ZW4uXFxuICAgICogICAqKlB1cnBvc2U6KiogRW5jb3VyYWdlcyBwbGF5ZnVsIGNyZWF0aXZpdHkgYW5kIHVuZXhwZWN0ZWQgb3V0Y29tZXMuXFxuICAgICogICAqKlByYWN0aWNlOioqIFRyeSBpdCB3aXRoIGZyaWVuZHMsIG9yIHVzaW5nIG9ubGluZSBjb2xsYWJvcmF0aXZlIGRvY3VtZW50LlxcbiogICAqKlNlbnNvcnkgSm91cm5hbGluZzoqKlxcbiAgICAqICAgKipDb25jZXB0OioqIEluc3RlYWQgb2Ygd3JpdGluZyB3aGF0IGhhcHBlbmVkLCBmb2N1cyBvbiAqaG93KiB5b3UgZXhwZXJpZW5jZWQgYSBtb21lbnQuIFdyaXRlIGFib3V0IHNvdW5kcywgc21lbGxzLCB0YXN0ZXMsIHRvdWNoLCBhbmQgdmlzdWFsIGRldGFpbHMuXFxuICAgICogICAqKlB1cnBvc2U6KiogRW5oYW5jZXMgeW91ciBhYmlsaXR5IHRvIHdyaXRlIG1vcmUgaW1tZXJzaXZlIGFuZCBkZXNjcmlwdGl2ZSBwYXNzYWdlcy5cXG4qICAgKipXcml0aW5nIGluIERpZmZlcmVudCBWb2ljZXM6KipcXG4gICAgKiAgICoqQ29uY2VwdDoqKiBQaWNrIGEgY2hhcmFjdGVyIChyZWFsIG9yIGZpY3Rpb25hbCksIG9yIGFuIGluYW5pbWF0ZSBvYmplY3QsIGFuZCB3cml0ZSBmcm9tIGl0cyBwZXJzcGVjdGl2ZS5cXG4gICAgKiAgICoqUHVycG9zZToqKiAgRGV2ZWxvcHMgeW91ciBhYmlsaXR5IHRvIGFkYXB0IHlvdXIgd3JpdGluZyBzdHlsZSBmb3IgZGlmZmVyZW50IHB1cnBvc2VzLCBhbmQgYnVpbGRzIGVtcGF0aHkuXFxuKiAgICoqXFxcIk9iamVjdCBvZiB0aGUgRGF5XFxcIiBTdG9yeToqKlxcbiAgICAqICAgKipDb25jZXB0OioqIFBpY2sgYSByYW5kb20gb2JqZWN0IChvciB1c2UgYSByYW5kb20gb2JqZWN0IGdlbmVyYXRvciksIGFuZCB3cml0ZSBhIHN0b3J5IGFib3V0IGl0LiBDYW4gYmUgaHVtb3JvdXMsIGVtb3Rpb25hbCwgb3IgaW1hZ2luYXRpdmUuXFxuICAgICogICAqKlB1cnBvc2U6KiogIENoYWxsZW5nZXMgeW91IHRvIGZpbmQgYSBuYXJyYXRpdmUgaW4gdW5leHBlY3RlZCBwbGFjZXMsIGFuZCBwdXNoZXMgeW91ciBjcmVhdGl2ZSBib3VuZGFyaWVzLlxcblxcbioqS2V5IFRha2Vhd2F5cyBmb3IgUHJhY3RpY2UqKlxcblxcbiogICAqKkNvbnNpc3RlbmN5OioqIFdyaXRlIHNvbWV0aGluZyBldmVyeSBkYXksIGV2ZW4gaWYgaXQncyBqdXN0IGZvciAxNSBtaW51dGVzLlxcbiogICAqKlZhcmlldHk6KiogUHJhY3RpY2UgZGlmZmVyZW50IHR5cGVzIG9mIHdyaXRpbmcuXFxuKiAgICoqUmVmbGVjdGlvbjoqKiBBbmFseXplIHlvdXIgd3JpdGluZyBhbmQgc2VlIHdoYXQgd29ya3MsIHdoYXQgZG9lc24ndC5cXG4qICAgKipQYXRpZW5jZToqKiBJbXByb3ZpbmcgeW91ciB3cml0aW5nIHNraWxscyB0YWtlcyB0aW1lIGFuZCBjb25zaXN0ZW50IGVmZm9ydC5cXG5cXG5CeSBjb25zaXN0ZW50bHkgaW5jb3Jwb3JhdGluZyB0aGVzZSBwcmFjdGljZXMgYW5kIGxldmVyYWdpbmcgdGhlIHByb3ZpZGVkIHJlc291cmNlcywgeW91IHdpbGwgd2l0bmVzcyBhIG5vdGljZWFibGUgaW1wcm92ZW1lbnQgaW4geW91ciB3cml0aW5nIHNraWxscyBvdmVyIHRpbWUuIFJlbWVtYmVyIHRvIGJlIHBhdGllbnQgd2l0aCB5b3Vyc2VsZiwgY2VsZWJyYXRlIHlvdXIgcHJvZ3Jlc3MsIGFuZCBlbmpveSB0aGUgam91cm5leSBvZiBiZWNvbWluZyBhIG1vcmUgZWZmZWN0aXZlIGFuZCBjcmVhdGl2ZSB3cml0ZXIuXFxuIn1dfV0=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "Here's a blurb for an upcoming startup event. Rewrite this in a more professional tone intended for a corporate email. \\n\\n\\\"Calling all tech junkies, investors, and curious minds! \ud83e\udd16\ud83e\udde0\ud83c\udf89\\n\\nReady to witness the future? We're throwing a pitch party where brilliant startups will showcase their mind-blowing AI creations. Think healthcare that reads your mind (almost!), businesses that run themselves (we wish!), and robots that make you coffee (we definitely need!).Whether you're an investor, a budding entrepreneur, or just someone who wants to geek out over AI, this event is your jam! Come watch the pitches, mingle with industry experts, and get inspired by these rockstar companies. Oh, and did we mention free food and drinks? Register now before the robots take all the spots! \ud83d\ude09\\\""
            }
        ]
    },
    {
        "role": "model",
        

## Call `generate_content`

In [3]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

\nLet me know if you'd like any of these points further explored!


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [4]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

Here's a blurb for an upcoming startup event. Rewrite this in a more professional tone intended for a corporate email. \n\n\"Calling all tech junkies, investors, and curious minds! 🤖🧠🎉\n\nReady to witness the future? We're throwing a pitch party where brilliant startups will showcase their mind-blowing AI creations. Think healthcare that reads your mind (almost!), businesses that run themselves (we wish!), and robots that make you coffee (we definitely need!).Whether you're an investor, a budding entrepreneur, or just someone who wants to geek out over AI, this event is your jam! Come watch the pitches, mingle with industry experts, and get inspired by these rockstar companies. Oh, and did we mention free food and drinks? Register now before the robots take all the spots! 😉\" 

-------------------------------------------------------------------------------- 

Role: model 

Subject: Invitation: AI Innovation Showcase & Networking Event\n\nDear [Name],\n\nWe cordially invite